<!-- Banner Image -->
<center>
    <img src="https://developer-blogs.nvidia.com/wp-content/uploads/2024/07/rag-representation.jpg" width="75%">
</center>

<!-- Links -->
<center>
  <a href="https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/workbench/" style="color: #76B900;">NVIDIA AI Workbench</a> •
  <a href="https://docs.nvidia.com/ai-workbench/" style="color: #76B900;">User Documentation</a> •
  <a href="https://docs.nvidia.com/ai-workbench/user-guide/latest/quickstart/example-projects.html" style="color: #76B900;">Example Projects Catalog</a> •
  <a href="https://forums.developer.nvidia.com/t/support-workbench-example-project-llama-3-finetune/303411" style="color: #76B900;"> Problem? Submit a ticket here! </a>
</center>

# Finetune and deploy an LLM model using SFT and VLLM 

Welcome!

In this notebook, we're going to walk through the flow of using supervised finetuning (SFT) on the Facebook opt-125m model from scratch using the base model and then deploying it using VLLM.  

Note that we will be using the base model in this notebook and not the instruct model. Additionally, we will be running through a full finetune with no quantization. This notebook was originally built for 2 A100-80GB GPUs, but the default hyperparameters have since been adjusted to run on 1x A100-80GB. If you're looking for a lighter Llama3-finetune, checkout out the other Llama3 finetuning notebook which uses Direct Preference Optimization.

#### Help us make this tutorial better! Please provide feedback on the [NVIDIA Developer Forum](https://forums.developer.nvidia.com/c/ai-data-science/nvidia-ai-workbench/671).

A note about running Jupyter Notebooks: Press Shift + Enter to run a cell. A * in the left-hand cell box means the cell is running. A number means it has completed. If your Notebook is acting weird, you can interrupt a too-long process by interrupting the kernel (Kernel tab -> Interrupt Kernel) or even restarting the kernel (Kernel tab -> Restart Kernel). Note restarting the kernel will require you to run everything from the beginning.

In [ ]:
#run HF model
!python -O -u -m vllm.entrypoints.openai.api_server \
    --host=0.0.0.0 \
    --port=8000 \
    --model=facebook/opt-125m \
    --tensor-parallel-size=1 \
    --gpu-memory-utilization=0.85 \
    --enforce-eager \
    --disable-frontend-multiprocessing \
    --max-model-len=2048

/opt/conda/lib/python3.10/site-packages/numpy/lib/scimath.py:46: RuntimeWarning: divide by zero encountered in log
  _ln2 = nx.log(2.0)
2025-01-19 13:58:56.229222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-19 13:58:56.636289: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-19 13:58:58.052483: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object f

# Open up a terminal and run

*Baseline test:*
curl http://localhost:8000/v1/completions \
   -H "Content-Type: application/json" \
   -d '{
       "model": "facebook/opt-125m",
       "prompt": "The god father movie is",
       "max_tokens": 30,
       "temperature": 0
   }'
